# Contacts4: Linked Data(1)

We gebruiken in de contact-documenten eigenschappen als `name` en `address`.
Door het schema en de validator weten we dat deze eigenschappen op een consistente manier gebruikt worden,
door alle documenten in de collection.

Het schema is beperkt tot de lokale database: andere contact-databases kunnen andere schema's gebruiken,
met andere namen en types voor de verschillende properties.
Dit maakt het uitwisselen van data en het koppelen van databases lastig.

*Linked data* probeert hiervoor een oplossing te bieden.
In zekere zin betekent *linked data* voor data in databases, wat het internet is voor computers (en toepassingen) in lokale netwerken.

* Voor het identificeren en adresseren van computers in het internet gebruik je het IP-adres, voor toepassingen aangevuld met het poortnummer.
* Voor *linked data* gebruik je URLs ("webadressen") voor identificatie en adressering.

De belangrijkste principes van Linked Data, zoals beschreven door Tim Berners Lee (de uitvinder van het web):

1. gebruik URIs als namen voor dingen;
2. gebruik HTTP URIs (d.w.z. URLs) zodat mensen de betekenis van deze namen op kunnen zoeken;
3. als iemand een URI opzoekt (via het web), geef dan zinvolle informatie;
4. gebruik hierin links (URLs) waarmee meer dingen ontdekt kunnen worden.

In eerste instantie richten we ons op (1) en (2).
De andere aspecten, in het bijzonder (4), behandelen we later.

## Namen van eigenschappen

De eerste stap is het gebruiken van URLs als namen (identificaties).
In het voorbeeld van de contacten-database betekent dit dat we URLs gebruiken als de namen van eigenschappen (properties, velden).
Begrippen als `address` en `telephone` zijn elders op het web gedefinieerd in gestandaardiseerde *schema's*.
Onder andere [schema.org](https://schema.org) heeft een aantal veelgebruikte begrippen in schema's beschreven,
bijvoorbeeld rond personen, organisaties en gebeurtenissen (events).

(Ook voor de personen en organisaties in de database willen we URLs gebruiken; daar komen we later op terug.)

```Python
person = {
    "http://schema.org/name": "Henk de Vries",
    "http://schema.org/telephone": "06 1234 5588"
}
```

De namen die we hier gebruiken voor de eigenschappen van een contact-document zijn URLs,
en bovendien kunnen we de betekenis van deze eigenschappen opzoeken, via de website van `schema.org`.

Een document met URLs als namen voor de eigenschappen is niet erg leesbaar.
De JSON-LD (JSON Linked Data) notatie biedt daarom een manier om deze notatie af te korten,
waarbij het principe van URLs als namen gehandhaafd blijft.
Voor deze afkorting voegen we aan een document (JSON-object) een *context* toe, 
met verkorte (lokale) namen in termen van globale URLs.

In [ ]:
person_context = {
        "name": "http://schema.org/name",
        "telephone": "http://schema.org/telephone"
    }
person = {
    "@context": person_context,
    "name": "Henk de Vries",
    "telephone": "06 1234 5588" 
}

*Door de context met de beschrijving van de namen expliciet aan het document toe te voegen,
wordt de betekenis van deze namen onafhankelijk van de context waarin het document gebruikt wordt.*

Met behulp van de pyld-library kunnen we een compacte vorm omzetten in een geëxpandeerde vorm:

In [ ]:
from pyld import jsonld
import json

expanded = jsonld.expand(person)
expanded

We kunnen een geëxpandeerde vorm ook omzetten in een compacte vorm:

In [ ]:
compacted = jsonld.compact(expanded, person_context)
compacted

## Types van document-eigenschappen

Naast de namen van de eigenschappen, kunnen we (als in een database-schema) ook de types van de eigenschappen beschrijven.
Voor het identificeren van een type gebruiken we dan ook weer een URI (URL).

In het geval van JSON-LD hebben we te maken met twee soorten types: *data types* en *object-types*.
Voorbeelden van data types zijn: Number, String, Date, ...
Voorbeelden van object types zijn: Person, Organization, Event, ...

We zullen beide in een voorbeeld uitwerken.

Vaak hebben we te maken met string-waarden die aan bepaalde voorwaarden voldoen.
Een string kan bijvoorbeeld een datum voorstellen.
(In JSON kunnen we dat niet uitdrukken; BSON heeft meer types, o.a. `Date` (datum en tijd).

> Een JSON-LD-context beschrijft de namen en types die gebruikt *kunnen* worden voor de eigenschappen van een document (object).
Een MongoDB-schema beschrijft de eigenschappen die *moeten* voorkomen in een document.
Deze twee concepten vullen elkaar aan.

(Wat is een geschikt voorbeeld? datum? - bijv. geboortedatum? id? Person? url?)

Voor het beschrijven van het type van een eigenschap gebruiken we `@type`; we moeten dan `@id` gebruiken voor de URL die we als naam van de eigenschap gebruiken.
In het geval van een URL is het type `@id`: dat is een web-id, ofwel een URL.

> Hoewel het type precies beschrijft wat het formaat is van de datum-string van `birthDate`, 
  namelijk een datum in ISO_8601-formaat, 
  hebben we hier geen mechanisme om het juiste gebruik van het formaat af te dwingen of te controleren.
  Het is in de eerste plaats een aanwijzing voor de lezer (mens of computer) van het person-document,
  om deze stringwaarde op de juiste manier te interpreteren.

**Opdracht**

* bekijk de definities van `http://schema.org/Person`, `http://schema.org/birthDate`, `http://schema.org/Date`

In [ ]:
person_context = {
        "name": "http://schema.org/name",
        "telephone": "http://schema.org/telephone",
        "homepage": {"@id": "http://schema.org/url", "@type": "@id"},
        "birthDate": {"@id": "http://schema.org/birthDate",
                      "@type": "http://schema.org/Date"}
    }
person = {
    "@context": person_context,
    "name": "Henk de Vries",
    "telephone": "06 1234 5588" ,
    "homepage": "http://henk.devries.org",
    "birthDate": "1970-1-1"
}

In [ ]:
expanded = jsonld.expand(person)
expanded

In [ ]:
compacted = jsonld.compact(expanded, person_context)
compacted

## Object type en object id

We hebben nu de mogelijkheden gebruikt om de namen en types van de eigenschappen van een document te beschrijven.
Een volgende stap is om het type van het document zelf te beschrijven.
Bijvoorbeeld: beschrijft dit document een persoon of een organisatie?

In [ ]:
person = {
    "http://schema.org/name": "Henk de Vries",
    "http://schema.org/telephone": "06 1234 5588"
}


In [ ]:
# expand a document, removing its context
# see: http://json-ld.org/spec/latest/json-ld/#expanded-document-form
expanded = jsonld.expand(compacted)
expanded

Je zien in deze expansie dat sommige eigenschappen, zoals "@type" en "birthDate"(???), meerdere waarden kunnen hebben. (Dit is standaard in de geëxpandeerde versie?)

### Object types

Met een object-type kunnen we het type van een document aangeven.
In de contacten-database kunnen we zo bijvoorbeeld verschil maken tussen *personen* en *organisaties*.
Deze hebben een deel van de eigenschappen gemeenschappelijk, maar hebben daarnaast ook eigenschappen die alleen voor dat type betekenis hebben.

> Een *object-type* is het type van een entiteit uit de wereld die je modelleert. Een *data-type* is een middel waarmee je dergelijke entiteiten modelleert.
Een eenvoudige eigenschap heeft gewoonlijk als type een *data-type* of een eenvoudige combinatie van data-types.
Een "complexe" eigenschap met als type een *object-type*, beschrijft gewoonlijk een *relatie* met een andere entiteit.


In [ ]:
context = {
    "xsd": "http://www.w3.org/2001/XMLSchema#",
    "person": "http://schema.org/Person",
    "birthDate": {"@id": "http://schema.org/birthDate", "@type": "xsd:date"},
    "name": "http://schema.org/name",
    "telephone": "http://schema.org/telephone",
    "homepage": {"@id": "http://schema.org/url", "@type": "@id"}    
}
person = {
    "@context": context,
    "@type": "person",
    "name": "Henk de Vries",
    "telephone": "06 1234 5588" ,
    "homepage": "http://henk.devries.org",
    "birthDate": "1970-1-1"    
}
jsonld.expand(person)

In [ ]:
jsonld.compact(jsonld.expand(person), context)

### XML data types

We hebben in dit voorbeeld voor de datum gebruik gemaakt van `http://www.w3.org/2001/XMLSchema#`.
Dit is de URI (URL?) voor XML Schema: deze definieert een groot aantal data-types.
Deze datatypes zijn onderdeel van die URL: de *prefix-notatie* `xsd:date` wordt geëxpandeerd tot `http://www.w3.org/2001/XMLSchema#date`. (Let ook op het `#` in de URL.) Als je deze URL raadpleegt krijg je niet direct de informatie over `string`: deze wordt in een ander document beschreven. Het is dus eigenlijk eerder een URI (identificatie) dan een URL (adres).

XML schema biedt veel meer mogelijkheden om data-types precies te beschrijven, bijvoorbeeld het bereik van getallen.

* zie o.a. https://www.w3.org/TR/xmlschema-2/#string
* en de figuur onder "3 Built-in types" op die pagina

In [ ]:
context = {
    "xsd": "http://www.w3.org/2001/XMLSchema#",
    "person": "http://schema.org/Person",
    "name": "http://schema.org/name",
    "birthDate": {"@id": "http://schema.org/birthDate", "@type": "xsd:date"},
    "telephone": "http://schema.org/telephone",
    "homepage": {"@id": "http://schema.org/url", "@type": "@id"},
    "birthDate": {"@id": "http://schema.org/birthDate",
                  "@type": "xsd:date"}
}
person = {
    "@context": context,
    "@type": "person",
    "name": "Henk de Vries",
    "telephone": "06 1234 5588" ,
    "homepage": "http://henk.devries.org",
    "birthDate": "1970-1-1"
}
jsonld.expand(person)

**Issue**

(Vraag: kunnen we deze prefixnotatie ook gebruiken voor `http://schema.org/`?
Is dat ongebruikelijk, of werkt dat niet zo?
Volgens mij werkt het wel; wordt dit ergens in de voorbeelden gebruikt?)

### Object ID

Een document vormt een *beschrijving* van een entiteit, bijvoorbeeld een persoon of een organisatie.
Het geeft je geen *identificatie* voor de entiteit.
In het web wordt als identificatie van een "resource" (de web-term voor entiteit) een URI (URL) gebruikt.

Als identificatie van een (natuurlijke) persoon kun je bijvoorbeeld de URL van zijn of haar persoonlijke webpagina gebruiken:
dat is in elk geval een unieke identificatie, en hopelijk ook een stabiele.

Een andere manier om een persoon te identificeren is aan het hand van het mail-adres.
Dit is persoonlijk, en verandert niet - ook als het niet meer in gebruik is: 
een persoonlijk mail-adres wordt niet aan een andere persoon gekoppeld.
Een mailadres heeft wel een nadeel: je wilt dit misschien niet publiek maken.
Een oplossing (gebruikt door FOAF - http://semanticweb.org/wiki/FOAF.html) is om een hashwaarde van het mailadres te gebruiken als identificatie (zie: http://xmlns.com/foaf/spec/#term_mbox_sha1sum).

Als identificatie van een organisatie kun je de homepage of het primaire domein van de organisatie gebruiken.

Wikidata geeft ook een ID voor sommige (bekende) personen.
Zie bijvoorbeeld:
    
* https://www.wikidata.org/wiki/Q12345
* https://www.wikidata.org/wiki/Q42
* https://www.wikidata.org/wiki/Q80

Ook voor auteurs van wetenschappelijke artikelen is er een dergelijke ID: zie bijvoorbeeld de verschillende IDs van 

Uit dit overzicht blijkt dan je meerdere ID-waarden kunt hebben voor een persoon of voor een organisatie.
Dat is niet zo'n probleem als je middelen hebt om de gelijkheid (equivalentie) van die IDs te bepalen (zoals bijvoorbeeld via Wikidata).

Merk op dat de "expanded" versie van de documenten hierboven altijd meerdere waarden voor een bepaalde eigenschap (attribuut, veld) toestaat.


## ToDo

* hoe gebruik je dit in combinatie met MongoDB?
* hoe gebruik je dit in het web?